<a href="https://colab.research.google.com/github/daedhaed/workout-calendar/blob/main/workout_analytcs_from_calendar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %% [markdown]
"""
# Google Calendar & Fitness Data Analysis Notebook

This notebook is divided into three sections:
1. **Configuration & Authentication**
   Set up and authenticate with Google Cloud and the Calendar API.
2. **Google Calendar Data Extraction**
   Fetch events from your Google Calendar.
3. **Integration with OpenAI for Fitness Data Analysis**
   Analyze your workout data using OpenAI's API.

**Before running:**
- Set up your Google Cloud project, enable the Calendar API, and create a service account.
- Download your service account credentials (JSON file).
- Share your calendar with the service account (if required).
- Secure your OpenAI API key (set as an environment variable or secure config).
- Replace placeholder values (e.g., `"YOUR_CALENDAR_ID"`, `"YOUR_OPENAI_API_KEY"`, `"path/to/credentials.json"`) with your own.
"""
# %% [code]
# Install required packages
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib openai

import os
import datetime
import csv
from google.colab import files, auth
from googleapiclient.discovery import build
from google.oauth2 import service_account
import openai

# %% [markdown]
"""
## Section 1: Configuration & Authentication

This section:
- Provides helper functions to upload the service account credentials.
- Authenticates the user and loads the credentials for the Calendar API.

Sensitive file paths and API keys are replaced by placeholders.
"""

# %% [code]
def upload_credentials():
    """
    Prompts the user to upload the service account credentials JSON file.
    Returns the filename of the uploaded file.
    """
    print("Please upload your service account credentials JSON file:")
    uploaded = files.upload()  # This opens a file upload dialog in Colab.
    for fn in uploaded.keys():
        print(f"Uploaded file: {fn} ({len(uploaded[fn])} bytes)")
        return fn
    return None

def authenticate_and_load_creds(credentials_path, scopes):
    """
    Authenticates the Colab user and loads the service account credentials.

    Args:
        credentials_path (str): Path to the credentials JSON file.
        scopes (list): List of scopes required for the API.

    Returns:
        Credentials object for Google APIs.
    """
    auth.authenticate_user()  # Authenticate the user.
    print("User authenticated successfully.")
    creds = service_account.Credentials.from_service_account_file(credentials_path, scopes=scopes)
    return creds

# %% [markdown]
"""
## Section 2: Google Calendar Data Extraction

This section:
- Builds a Calendar API service.
- Fetches a list of upcoming events from your calendar.
- **IMPORTANT:** Replace `"YOUR_CALENDAR_ID"` with your own calendar identifier.
"""

# %% [code]
# Define Calendar API scope (read-only in this example)
CALENDAR_SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']

# Define the default credentials file path.
# If the file is not found at the default location, prompt the user to upload it.
DEFAULT_CREDENTIALS_PATH = "path/to/credentials.json"  # Replace with your default path if desired.
if not os.path.exists(DEFAULT_CREDENTIALS_PATH):
    DEFAULT_CREDENTIALS_PATH = upload_credentials()

# Authenticate and load credentials
creds = authenticate_and_load_creds(DEFAULT_CREDENTIALS_PATH, CALENDAR_SCOPES)

# Build the Calendar service
calendar_service = build('calendar', 'v3', credentials=creds)

# Replace with your own calendar ID (this might be your email address or a specific calendar ID)
YOUR_CALENDAR_ID = "YOUR_CALENDAR_ID"  # <-- Update this placeholder

# Example: List the next 10 upcoming events
now = datetime.datetime.utcnow().isoformat() + 'Z'  # 'Z' indicates UTC time
events_result = calendar_service.events().list(
    calendarId=YOUR_CALENDAR_ID,
    timeMin=now,
    maxResults=10,
    singleEvents=True,
    orderBy='startTime'
).execute()
events = events_result.get('items', [])

if not events:
    print("No upcoming events found.")
else:
    print("Upcoming events:")
    for event in events:
        start = event.get('start', {}).get('dateTime', event.get('start', {}).get('date'))
        summary = event.get('summary', 'No Title')
        print(f"{start}: {summary}")

# %% [markdown]
"""
## Section 3: Integration with OpenAI for Fitness Data Analysis

This section:
- Defines a helper function to analyze workout data using OpenAI's API.
- Loads workout data from a CSV file (or uses sample data if the file is absent).
- **IMPORTANT:** Securely set your OpenAI API key (e.g., via an environment variable).
"""

# %% [code]
def analyze_fitness_data(workout_data, openai_api_key):
    """
    Uses OpenAI's API to analyze fitness data.

    Args:
        workout_data (str): A string representation of the workout CSV data.
        openai_api_key (str): Your OpenAI API key.

    Returns:
        A string with the analysis results from the OpenAI model.
    """
    # Set the OpenAI API key (ensure this is set securely)
    openai.api_key = openai_api_key

    prompt_text = f"""
You are an advanced fitness data analyst. I have a table of workouts, each row describing an exercise session.
Please do the following:

1. Classify each workout as either:
   - Weightlifting
   - Triathlon Training (running, cycling, or swimming).

2. For Weightlifting workouts:
   - Parse exercises, sets, reps, and weight if provided.
   - Summarize total sets/reps if possible.

3. For Triathlon workouts:
   - Identify which discipline (running, cycling, swimming).
   - Note distance, time, or pace if provided.

4. Output a structured summary of each workout in JSON format, with relevant fields.
5. Provide any additional insights or patterns you notice.

Here is the workout data:
{workout_data}
"""
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt_text}],
        temperature=0
    )
    return response.choices[0].message.content

# Load workout data from a CSV file, or use sample data if not available.
WORKOUT_CSV_PATH = "workout_data.csv"  # Replace with your file path if applicable.
if os.path.exists(WORKOUT_CSV_PATH):
    with open(WORKOUT_CSV_PATH, "r", encoding="utf-8") as f:
        workout_data = f.read()
else:
    workout_data = """Summary,Start
Example Workout 1,2025-01-01T00:00:00Z
Example Workout 2,2025-01-02T00:00:00Z
"""

# Retrieve OpenAI API key from an environment variable (or replace with your key securely)
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY", "YOUR_OPENAI_API_KEY")  # <-- Update securely!

# Analyze the fitness data using OpenAI
analysis_result = analyze_fitness_data(workout_data, OPENAI_API_KEY)
print("Fitness Data Analysis Result:")
print(analysis_result)

# %% [markdown]
"""
# End of Notebook

Ensure you replace all placeholder values with your actual credentials, calendar ID, and API key before publishing.
"""